In [1]:
import cv2
target_file = './resource/test/fail2_issue_0.png'
img = cv2.imread(target_file)

In [21]:
from sklearn.cluster import KMeans
import numpy as np

def _hsv_color( component_img: np.ndarray):
    if len(component_img.shape) == 3:
        img_rgb = cv2.cvtColor(component_img, cv2.COLOR_BGR2RGB)
    else:
        img_rgb = component_img

    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)

    # h channel main color
    h_channel = img_hsv[:, :, 0]
    hist_h = cv2.calcHist([h_channel], [0], None, [180], [0, 180])

    # find peaks
    peaks = []
    for i in range(1, len(hist_h) - 1):
        if hist_h[i] > hist_h[i-1] and hist_h[i] > hist_h[i+1] and hist_h[i] > np.max(hist_h) * 0.1:
            peaks.append(i)
    
    # select peaks
    peaks = sorted(peaks, key=lambda x: hist_h[x], reverse=True)[:3]

    # find colors
    colors = []
    for peak_h in peaks:
        mask = np.abs(h_channel - peak_h) < 10
        if np.any(mask):
            masked_pixels = img_rgb[mask]
            if len(masked_pixels) > 0:
                avg_color = np.mean(masked_pixels, axis=0).astype(int)
                colors.append(avg_color.tolist())
    
    # HSV에서 충분한 색상을 찾지 못한 경우 KMeans로 보완
    if len(colors) < 3:
        pixels_reshaped = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        kmeans.fit(pixels_reshaped)

        for center in kmeans.cluster_centers_:
            if len(colors) < 3:
                colors.append(center.astype(int).tolist())
    
    return colors[:3]

def _lab_color( component_img: np.ndarray):
    if len(component_img.shape) == 3:
        img_rgb = cv2.cvtColor(component_img, cv2.COLOR_BGR2RGB)
    else:
        img_rgb = component_img

    # lab
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)

    # a, b channel 
    a_channel = img_lab[:, :, 1]
    b_channel = img_lab[:, :, 2]

    # histogram
    hist_ab = cv2.calcHist([a_channel, b_channel], [0, 1], None, [32, 32], [0, 256, 0, 256])

    # find peaks
    peaks = []
    threshold = np.max(hist_ab) * 0.05

    for i in range(1, hist_ab.shape[0] - 1):
        for j in range(1, hist_ab.shape[1] - 1):
            if (hist_ab[i, j] > threshold and 
                hist_ab[i, j] > hist_ab[i-1, j] and hist_ab[i, j] > hist_ab[i+1, j] and
                hist_ab[i, j] > hist_ab[i, j-1] and hist_ab[i, j] > hist_ab[i, j+1]):
                peaks.append((i, j, hist_ab[i, j]))

    peaks = sorted(peaks, key=lambda x: x[2], reverse=True)[:3]

    #find colors
    colors = []
    for a_bin, b_bin, _ in peaks:
        a_center = a_bin * 8
        b_center = b_bin * 8

        a_mask = np.abs(a_channel - a_center) < 12
        b_mask = np.abs(b_channel - b_center) < 12
        region_mask = a_mask & b_mask

        if np.any(region_mask):
            region_pixels = img_rgb[region_mask]
            avg_color = np.mean(region_pixels, axis=0).astype(int)
            colors.append(avg_color.tolist())

    # LAB에서 충분한 색상을 찾지 못한 경우 KMeans로 보완
    if len(colors) < 3:
        pixels_reshaped = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        kmeans.fit(pixels_reshaped)

        for center in kmeans.cluster_centers_:
            if len(colors) < 3:
                colors.append(center.astype(int).tolist())

    return colors[:3]

In [23]:
import numpy as np

def color_extraction(component_img: np.ndarray):
    if len(component_img.shape) == 3:
        img_rgb = cv2.cvtColor(component_img, cv2.COLOR_BGR2RGB)
    else:
        img_rgb = component_img
    
    # 이미지 특성 분석
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(img_hsv)

    # 고채도 픽셀 비율
    high_saturation_ratio = np.sum(s > 50) / s.size

    print(f"고채도 픽셀 비율: {high_saturation_ratio}")

    if high_saturation_ratio < 0.3:
        print("LAB 색공간 사용")
        colors = _lab_color(img_rgb)
    else:
        print("HSV 색공간 사용")
        colors = _hsv_color(component_img)
    
    print(f"추출된 색상 개수: {len(colors)}")
    return colors

In [24]:
color_extraction(img)

고채도 픽셀 비율: 0.9037339226018472
HSV 색공간 사용
추출된 색상 개수: 3


[[93, 66, 42], [88, 64, 40], [114, 79, 50]]

In [29]:
from paddleocr import PaddleOCR

paddle_ocr = PaddleOCR(
        det_model_dir='./src/weights/en_PP-OCRv3_det_infer',
        rec_model_dir='./src/weights/en_PP-OCRv3_rec_infer', 
        cls_model_dir='./src/weights/ch_ppocr_mobile_v2.0_cls_infer',
        lang='en',
        use_angle_cls=False,
        use_gpu=False,
        show_log=False)

def _extract_darkest_color(component_img: np.ndarray) -> tuple:
    hsv = cv2.cvtColor(component_img, cv2.COLOR_BGR2HSV)
    dark_color = np.min(hsv, axis=0)
    return dark_color

def _extract_brightest_color(component_img: np.ndarray) -> tuple:
    hsv = cv2.cvtColor(component_img, cv2.COLOR_BGR2HSV)
    bright_color = np.max(hsv, axis=0)
    return bright_color

def _color_diff(color1: tuple, color2: tuple) -> float:
    return np.linalg.norm(np.array(color1) - np.array(color2))

def extract_text_color(img: np.ndarray):
    text = paddle_ocr.ocr(img, cls=False)
    print(text)

    if text is None or len(text) == 0:
        dark = _extract_darkest_color(img)
        bright = _extract_brightest_color(img)
        diff = _color_diff(dark, bright)
        print(f"text_color: {dark}, {bright}, {diff}")
        return diff

    text_pixels = []

    for item in text:
        coords, (text, confidence) = item
        if confidence > 0.6:
            coords_array = np.array(coords, dtype=np.int32)
            text_bbox = cv2.boundingRect(coords_array)
            tx, ty, tw, th = text_bbox
            text_region = img[ty:ty+th, tx:tx+tw]
            text_pixel = text_region.reshape(-1, 3)
            text_pixels.append(text_pixel)

    if len(text_pixels) == 0:
        dark = _extract_darkest_color(img)
        bright = _extract_brightest_color(img)
        diff = _color_diff(dark, bright)
        print(f"text_color: {dark}, {bright}, {diff}")
        return diff

    text_pixels = np.array(text_pixels)
    text_color = color_extraction(text_pixels)

    return text_color












In [30]:
extract_text_color(img)

[None]


TypeError: cannot unpack non-iterable NoneType object